## **Notebook Destinado a Machine Learning**

In [1]:
import os

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import matplotlib.pyplot as plt

#### Carregando os dados

In [2]:
load_dotenv()
def load_data():
    DB_USER = os.getenv("POSTGRES_USER")
    DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")
    DB_HOST = os.getenv("POSTGRES_HOST")
    DB_PORT = os.getenv("POSTGRES_PORT")
    DB_NAME = os.getenv("POSTGRES_DB") 

    connection_str = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(connection_str)

    query = """
    SELECT
        acidentes.id, 
        classificacao_acidente,
        regiao,
        tipo_acidente,
        causa_acidente,
        DATE_PART('HOUR', data_completa) AS hora,
        uso_solo,
        br,
        tipo_veiculo
    FROM relational.acidentes
    INNER JOIN relational.envolvidos ON acidentes.id = envolvidos.id_acidente"""


df = load_data()

In [3]:
# Removendo a coluna id
df.drop(columns=['id'], inplace=True)

In [4]:
df.shape

(662962, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662962 entries, 0 to 662961
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   classificacao_acidente  662962 non-null  object 
 1   regiao                  662962 non-null  object 
 2   tipo_acidente           662962 non-null  object 
 3   causa_acidente          662962 non-null  object 
 4   hora                    662962 non-null  float64
 5   uso_solo                662962 non-null  object 
 6   br                      662962 non-null  object 
 7   tipo_veiculo            662962 non-null  object 
dtypes: float64(1), object(7)
memory usage: 40.5+ MB


**Fazendo Feature Engineering, para melhorar a performance do modelo**

In [8]:
# Escolhendo as causas com pelo menos 5000 registros

causas_comuns = df['causa_acidente'].value_counts()[df['causa_acidente'].value_counts() > 5000].index
df = df[df['causa_acidente'].isin(causas_comuns)]
df['causa_acidente'].value_counts()

causa_acidente
Reação tardia ou ineficiente do condutor                     118275
Velocidade Incompatível                                       58185
Falta de Atenção à Condução                                   53778
Acessar a via sem observar a presença dos outros veículos     42634
Condutor deixou de manter distância do veículo da frente      42022
Ingestão de Álcool                                            39587
Manobra de mudança de faixa                                   29377
Desobediência às normas de trânsito pelo condutor             23266
Defeito Mecânico no Veículo                                   22682
Condutor Dormindo                                             19456
Ultrapassagem Indevida                                        18046
Transitar na contramão                                        17111
Chuva                                                         11876
Desrespeitar a preferência no cruzamento                      11868
Não guardar distância de seguranç

In [7]:
# Englobando causas semelhantes

df.loc[df['causa_acidente'] == 'Ausência de reação do condutor', 'causa_acidente'] = 'Reação tardia ou ineficiente do condutor'
df.loc[df['causa_acidente'] == 'Ingestão de álcool pelo condutor', 'causa_acidente'] = 'Ingestão de Álcool'
df.loc[df['causa_acidente'] == 'Demais falhas mecânicas ou elétricas', 'causa_acidente'] = 'Defeito Mecânico no Veículo'

In [9]:
# Escolhendo as rodovias com pelo menos 1000 registros

maiores_br = df['br'].value_counts()[df['br'].value_counts() > 1000].index
df = df[df['br'].isin(maiores_br)]
df['br'].value_counts()

br
BR-101    91817
BR-116    84494
BR-40     26769
BR-381    26180
BR-153    22298
BR-163    19700
BR-364    17676
BR-277    16951
BR-376    14461
BR-262    13798
BR-230    12857
BR-470    12244
BR-316    10386
BR-282    10214
BR-60      8265
BR-70      8053
BR-20      8032
BR-232     6718
BR-369     6714
BR-324     6674
BR-280     6528
BR-50      6299
BR-222     6077
BR-290     6025
BR-476     6007
BR-386     5952
BR-158     5695
BR-135     5441
BR-365     5294
BR-343     4662
BR-174     4086
BR-10      4002
BR-285     3623
BR-104     3434
BR-251     3344
BR-242     3130
BR-392     2935
BR-304     2879
BR-110     2536
BR-393     2496
BR-235     2417
BR-407     2397
BR-267     2277
BR-373     2145
BR-423     2045
BR-226     1901
BR-493     1805
BR-319     1593
BR-356     1580
BR-428     1438
BR-414     1100
BR-465     1076
BR-459     1032
BR-287     1031
BR-408     1022
BR-210     1020
BR-259     1017
BR-80      1011
BR-472     1002
Name: count, dtype: int64

## Treinando o modelo de Classificação

**Usando o XGBoost**

Escolhi o XGBoost, pois é um dos algoritmos mais eficientes para classificação, utilizando paralelismo e otimização de recursos. Além disso, ele é muito eficiente para lidar com dados desbalanceados, encontrando padrões e relações entre as variáveis, possibilitando uma melhor performance do modelo.

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [11]:
X = df.drop(columns=['classificacao_acidente'])
y = df['classificacao_acidente']

In [12]:
categorical_features = X.select_dtypes(include="object").columns
numerical_features = X.select_dtypes(exclude="object").columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [13]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.33, random_state=42)

In [15]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        objective='multi:softmax',
        eval_metric='mlogloss'
    ))
])


In [22]:
param_grid = {
    'classifier__n_estimators': [200, 500, 700, 1000],
    'classifier__max_depth': [3, 6, 9, 12],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__subsample': [0.6, 0.9, 1.0],
    'classifier__colsample_bytree': [0.8, 0.9, 1.0],
    'classifier__gamma': [0.1, 0.3, 0.5],
    'classifier__reg_alpha': [0, 0.1, 1],
    'classifier__reg_lambda': [1, 10, 100]
}

In [23]:
randomized_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=50, cv=5, 
                                       verbose=1, n_jobs=4, scoring='accuracy', random_state=42)
randomized_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/home/saulo/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               Index(['hora', 'latitude', 'longitude'], dtype='object')),
                                                                              ('cat',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               Index(['regiao', 'tipo_acidente', 'causa_acidente', 'uso_solo', 'br',
       'tipo_veiculo'],
      dtype='object'))])),
                                             ('classifier',
                                              XGBClassifie...
                   param_distributions={'classifier__colsample_bytree': [0.8,
                                                                         0.9,
                                                                         1.0],
                                        'classifier__gamma': [0.1, 0.3, 0.5],
                                        'classifier__learning_rate': [0.01, 0.1,
                                                                      0.2],
                                        'classifier__max_depth': [3, 6, 9, 12],
                                        'classifier__n_estimators': [200, 500,
                                                                     700,
                                                                     1000],
                                        'classifier__reg_alpha': [0, 0.1, 1],
                                        'classifier__reg_lambda': [1, 10, 100],
                                        'classifier__subsample': [0.6, 0.9,
                                                                  1.0]},
                   random_state=42, scoring='accuracy', verbose=1)

In [27]:
print(f"Melhores parâmetros: {randomized_search.best_params_}")
print(f"Melhor acurácia: {randomized_search.best_score_:.2f}%")

Melhores parâmetros: {'classifier__subsample': 0.6, 'classifier__reg_lambda': 10, 'classifier__reg_alpha': 0.1, 'classifier__n_estimators': 700, 'classifier__max_depth': 12, 'classifier__learning_rate': 0.2, 'classifier__gamma': 0.5, 'classifier__colsample_bytree': 0.9}
Melhor acurácia: 0.85%


In [22]:
import joblib

joblib.dump(randomized_search.best_estimator_, 'modelo_treinado.pkl')

['model.pkl']